# Train and Apply Models

In [ ]:
from ML.model_training import (
    omit_patient_video,
    single_user_split,
    train_lstm,
    train_random_forest,
    build_decod_lstm_sequences,
)
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    confusion_matrix,
    classification_report,
)
import numpy as np
import pandas as pd
from ML import utils
import sys
import random
from itertools import product

In [ ]:
# remove_list = [0, 1, 2, 4, 5, 8, 9, 10, 11, 14, 16, 17, 18, 22]
remove_list = []

def balance(X, y, seed=5):
    c = y.value_counts()
    if c.get("high", 0) == c.get("low", 0):
        return X.reset_index(drop=True), y.reset_index(drop=True)
    maj = c.idxmax()
    m = c.min()
    keep = y[y != maj].index.union(y[y == maj].sample(m, random_state=seed).index)
    return X.loc[keep].reset_index(drop=True), y.loc[keep].reset_index(drop=True)

# subjects = [i for i in range(0, 23)]
# subjects = [3]

## LSTM CV Optimizer - PSD features only

This is currently running LOSO CV, by removing 18 videos (all) from each user.

In [ ]:
CV_SIZE = 5
subjects = []
while len(subjects) < 9:
    subject = random.randint(0, 22)
    if subject not in subjects:
        subjects.append(subject)

param_grid = {
    "lr": [0.0001],
    "epochs": [100],
    "units": [128, 256],
    "batch_size": [64],
    "timesteps": [76],
}

best_params = None
best_mean_acc = -float("inf")

print("Starting global hyperparameter search...\n")

for lr, epochs, units, batch_size, timesteps in product(
    param_grid["lr"],
    param_grid["epochs"],
    param_grid["units"],
    param_grid["batch_size"],
    param_grid["timesteps"],
):
    combo_accs = []
    for i in subjects:
        relabel = False
        while True:
            while True:
                X_train_df, X_test_df, arousal_train, arousal_test = omit_patient_video(
                    target="arousal",
                    selected_user=i,
                    trials=18,
                    # holdout_videos=[2, 10, 15],
                    exclude_users=remove_list,
                    filename="datasets/experiment_features_table.csv",
                    relabel=relabel,
                )

                features = utils.filter_features(
                    X_train_df.columns,
                    remove_bands=["gamma"],
                )
                features = [c for c in features if c in X_train_df.columns]
                features.remove("arousal")
                features.remove("valence")
                features.remove("patient_index")
                features.remove("video_index")

                # Build sequence-level data (trials = (patient, video))
                X_train_seq, y_train_seq = build_lstm_sequences(
                    X_train_df,
                    features,
                    target_col="arousal",
                    thresh=3.8,
                    fixed_T=timesteps,
                )
                X_test_seq, y_test_seq = build_lstm_sequences(
                    X_test_df,
                    features,
                    target_col="arousal",
                    thresh=3.8,
                    fixed_T=timesteps,
                )

                minority = (y_train_seq == 1.0).sum() < (y_train_seq == 0.0).sum()
                maj_label = 0.0 if minority else 1.0
                min_label = 1.0 - maj_label
                n_pos = (y_train_seq == 1.0).sum()
                n_neg = (y_train_seq == 1.0).sum()

                idx_maj = np.where(y_train_seq == maj_label)[0]
                idx_min = np.where(y_train_seq == min_label)[0]

                if len(idx_min) == 0:
                    # no samples of one class, redo split
                    print("No minority class in this split, re-drawing...")
                    continue

                # reps = int(np.ceil(len(idx_maj) / len(idx_min)))
                # idx_min_upsampled = np.tile(idx_min, reps)[: len(idx_maj)]

                # idx_balanced = np.concatenate([idx_maj, idx_min_upsampled])
                # np.random.shuffle(idx_balanced)

                # X_train_bal = X_train_seq[idx_balanced]
                # y_train_bal = y_train_seq[idx_balanced]

                print("y_train counts:", np.bincount(y_train_seq.astype(int)))
                print("y_test counts:", np.bincount(y_test_seq.astype(int)))

                break

            lstm, X_test_eval, y_test_eval = train_lstm(
                X_train_seq,
                X_test_seq,
                y_train_seq,
                y_test_seq,
                lr=lr,
                epochs=epochs,
                units=units,
                batch_size=batch_size,
                dropout=0.4,
                recurrent_dropout=0,
                bidirectional=True,
            )
            y_prob = lstm.predict(X_test_eval).ravel()
            arousal_pred = (y_prob >= 0.5).astype(int) # TODO

            acc = accuracy_score(y_test_eval, arousal_pred)
            print(acc)
            relabel = False
            break

        combo_accs.append(float(acc))

        print("\nConfusion Matrix (subject):")
        print(confusion_matrix(y_test_eval, arousal_pred))
    mean_acc = float(np.mean(combo_accs))
    print(
        f"Params lr={lr}, epochs={epochs}, units={units}, batch_size={batch_size}, timesteps={timesteps} "
        f"-> mean acc across subjects = {mean_acc:.4f}"
    )

    if mean_acc > best_mean_acc:
        best_lstm = lstm
        best_mean_acc = mean_acc
        best_params = {
            "lr": lr,
            "epochs": epochs,
            "units": units,
            "batch_size": batch_size,
            "timesteps": timesteps,
        }

best_lstm.save("models/transfer_model.keras")


print("\nBest universal hyperparameters:")
print(256)
print(f"Mean accuracy across subjects (tuning): {best_mean_acc:.4f}\n")

### Fine-tune for User